In [1]:
import numpy as np 
import pandas as pd 
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt 
import requests


# Load Realtime API Stock Data

In [2]:
def get_stock_data(ticker, start_date, end_date):
    api_key = '50dbca4a8576228510cb129fe07db8e9a178bb9b'
    base_url = (f'https://api.tiingo.com/tiingo/daily/{ticker}/prices?')
    payload = {
        'token': api_key, 
        'startDate': start_date,
        'endDate': end_date
    }
#     data_response = requests.get(base_url, params=payload)
#     json_data = data_response.json()
#     df = json_normalize(response)
#     elevations = response.read()
#     data = json.loads(str(response))
#     df = pd.json_normalize(data)
#     df = pd.json_normalize(json_data)
#     df = df[['date', 'close']]
#     df.reset_index(level=0, inplace=True)

    response = requests.get(base_url, params=payload)
    
    
  #  df = pd.DataFrame.from_dict(response, orient="index")
    
    
#     dates = []
#     closes = []
    
#     for json_object in response:
#         date = json_object['date']
#         close = json_object['close']
        
#         dates.append(date)
#         closes.append(close)

#     df = json_normalize(dates)
#     df = pd.DataFrame(closes).transpose()
    return response
    
# panel_data = data.DataReader('INPX', 'google', start_date, end_date)

In [16]:
data_response = get_stock_data('NIO', '2020-06-01', '2020-11-04')
# print(data_response.url)
print(data_response.json())

[{'date': '2020-06-01T00:00:00.000Z', 'close': 4.26, 'high': 4.33, 'low': 3.96, 'open': 4.0, 'volume': 68951059, 'adjClose': 4.26, 'adjHigh': 4.33, 'adjLow': 3.96, 'adjOpen': 4.0, 'adjVolume': 68951059, 'divCash': 0.0, 'splitFactor': 1.0}, {'date': '2020-06-02T00:00:00.000Z', 'close': 4.7, 'high': 4.73, 'low': 4.35, 'open': 4.36, 'volume': 81717678, 'adjClose': 4.7, 'adjHigh': 4.73, 'adjLow': 4.35, 'adjOpen': 4.36, 'adjVolume': 81717678, 'divCash': 0.0, 'splitFactor': 1.0}, {'date': '2020-06-03T00:00:00.000Z', 'close': 5.6, 'high': 5.7, 'low': 4.98, 'open': 5.0, 'volume': 198303797, 'adjClose': 5.6, 'adjHigh': 5.7, 'adjLow': 4.98, 'adjOpen': 5.0, 'adjVolume': 198303797, 'divCash': 0.0, 'splitFactor': 1.0}, {'date': '2020-06-04T00:00:00.000Z', 'close': 5.97, 'high': 6.2, 'low': 5.61, 'open': 6.01, 'volume': 172617852, 'adjClose': 5.97, 'adjHigh': 6.2, 'adjLow': 5.61, 'adjOpen': 6.01, 'adjVolume': 172617852, 'divCash': 0.0, 'splitFactor': 1.0}, {'date': '2020-06-05T00:00:00.000Z', 'close

In [17]:
def create_datasets(ticker, start_date, end_date):
    json_data = data_response.json()
    
    dates = []
    highs = []
    lows = []
    opens = []
    closes = []
    vols = []
    
    for json_object in json_data:
        date = json_object['date']
        high = json_object['high']
        low = json_object['low']
        opened = json_object['open']
        close = json_object['adjClose']
        vol = json_object['volume']
        
        dates.append(date)
        highs.append(high)
        lows.append(low)
        opens.append(opened)
        closes.append(close)
        vols.append(vol)
    
    
    df = pd.DataFrame(list(zip(dates, closes)), columns=['Dates', 'adjClose']) 
    
    return df

In [18]:
# df = pd.read_csv('Data/LMND.csv')
# df.tail()
# # Get the Adjusted Close Price 
# df = df[['Adj Close']] 
# # Take a look at the new data 
# print(df.head())

## Create DataFrame

In [19]:
df = create_datasets('NIO', '2020-06-01', '2020-11-04')
#x_test, y_test = create_datasets('LMND', '2020-06-01', '2020-10-31')

df.tail()

,Dates,adjClose
105,2020-10-28T00:00:00.000Z,27.51
106,2020-10-29T00:00:00.000Z,31.99
107,2020-10-30T00:00:00.000Z,30.58
108,2020-11-02T00:00:00.000Z,33.32
109,2020-11-03T00:00:00.000Z,35.50


In [20]:
# df.tail()
# df['Dates'] = df['Dates'][0][:-14]
# api_df[:-5]
df['Ndates'] = df.Dates.str[:-14]
# df['NDates'] = df.Dates.apply(lambda x: x[:-14])
df.drop('Dates', inplace=True, axis=1)
df['Ndates']

0      2020-06-01
1      2020-06-02
2      2020-06-03
3      2020-06-04
4      2020-06-05
          ...    
105    2020-10-28
106    2020-10-29
107    2020-10-30
108    2020-11-02
109    2020-11-03
Name: Ndates, Length: 110, dtype: object

In [21]:
for label, content in df.items():
    if pd.api.types.is_string_dtype(content):
        df[label] = content.astype("category").cat.as_ordered()

In [22]:
df['Ndates']

0      2020-06-01
1      2020-06-02
2      2020-06-03
3      2020-06-04
4      2020-06-05
          ...    
105    2020-10-28
106    2020-10-29
107    2020-10-30
108    2020-11-02
109    2020-11-03
Name: Ndates, Length: 110, dtype: category
Categories (110, object): ['2020-06-01' < '2020-06-02' < '2020-06-03' < '2020-06-04' ... '2020-10-29' < '2020-10-30' < '2020-11-02' < '2020-11-03']

In [23]:
df['Ndates'] = pd.to_datetime(df['Ndates'])

df['Ndates']

0     2020-06-01
1     2020-06-02
2     2020-06-03
3     2020-06-04
4     2020-06-05
         ...    
105   2020-10-28
106   2020-10-29
107   2020-10-30
108   2020-11-02
109   2020-11-03
Name: Ndates, Length: 110, dtype: datetime64[ns]

In [24]:
# A variable for predicting 'n' days out into the future
forecast_out = 3 #'n=30' days
#Create another column (the target ) shifted 'n' units up
df['Prediction'] = df[['adjClose']].shift(-forecast_out)
#print the new data set
df.tail()

,adjClose,Ndates,Prediction
105,27.51,2020-10-28,33.32
106,31.99,2020-10-29,35.50
107,30.58,2020-10-30,NaN
108,33.32,2020-11-02,NaN
109,35.50,2020-11-03,NaN


In [25]:
df['Ndates'] = pd.to_numeric(df.Ndates, downcast='float')

In [26]:
### Create the independent data set (X)  #######
# Convert the dataframe to a numpy array
X = np.array(df.drop(['Prediction'],1))

#Remove the last '30' rows
X = X[:-forecast_out]
print(X)

[[4.26000000e+00 1.59096955e+18]
 [4.70000000e+00 1.59105600e+18]
 [5.60000000e+00 1.59114245e+18]
 [5.97000000e+00 1.59122876e+18]
 [5.59000000e+00 1.59131521e+18]
 [5.97000000e+00 1.59157442e+18]
 [6.68000000e+00 1.59166087e+18]
 [6.30000000e+00 1.59174718e+18]
 [5.91000000e+00 1.59183363e+18]
 [6.10000000e+00 1.59191994e+18]
 [6.83000000e+00 1.59217915e+18]
 [6.99000000e+00 1.59226560e+18]
 [6.84000000e+00 1.59235205e+18]
 [7.18000000e+00 1.59243836e+18]
 [7.34000000e+00 1.59252481e+18]
 [7.43000000e+00 1.59278402e+18]
 [7.23000000e+00 1.59287047e+18]
 [6.86000000e+00 1.59295678e+18]
 [7.08000000e+00 1.59304323e+18]
 [6.90000000e+00 1.59312954e+18]
 [7.23000000e+00 1.59338875e+18]
 [7.72000000e+00 1.59347520e+18]
 [7.91000000e+00 1.59356165e+18]
 [9.38000000e+00 1.59364796e+18]
 [1.15100000e+01 1.59399362e+18]
 [1.32200000e+01 1.59408007e+18]
 [1.28800000e+01 1.59416638e+18]
 [1.45700000e+01 1.59425283e+18]
 [1.49800000e+01 1.59433914e+18]
 [1.38400000e+01 1.59459835e+18]
 [1.409000

In [27]:
### Create the dependent data set (y)  #####
# Convert the dataframe to a numpy array 
y = np.array(df['Prediction'])
# Get all of the y values except the last '30' rows
y = y[:-forecast_out]
print(y)

[ 5.97  5.59  5.97  6.68  6.3   5.91  6.1   6.83  6.99  6.84  7.18  7.34
  7.43  7.23  6.86  7.08  6.9   7.23  7.72  7.91  9.38 11.51 13.22 12.88
 14.57 14.98 13.84 14.09 13.52 12.94 11.09 12.82 12.88 12.38 12.15 11.82
 11.69 12.27 12.7  12.2  11.94 13.6  13.64 13.92 13.84 13.42 14.21 12.99
 13.36 13.36 13.1  14.05 14.3  14.06 13.78 14.12 14.97 17.84 20.44 19.88
 18.5  19.03 20.12 19.9  18.7  17.98 17.03 18.09 17.74 17.97 18.73 19.19
 19.01 19.3  19.41 18.81 19.23 17.96 17.85 18.32 18.81 20.85 21.22 21.76
 21.18 21.59 20.67 21.56 21.44 21.47 21.85 21.62 26.5  28.07 28.48 27.63
 27.96 27.86 27.38 27.16 26.01 28.44 27.51 31.99 30.58 33.32 35.5 ]


# Split Data into Train & Test

In [28]:
# Split the data into 80% training and 20% testing
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

# Create Regression Models

In [29]:
# random forest regressor 
from sklearn.ensemble import RandomForestRegressor

rfr = RandomForestRegressor(max_depth=3,random_state=0)
rfr.fit(x_train, y_train)

RandomForestRegressor(max_depth=3, random_state=0)

In [30]:
np.random.seed(42)
rfr_confidence = rfr.score(x_test, y_test)
print("rfr confidence", rfr_confidence)

rfr confidence 0.9318650236270147


In [31]:
# Create and train the Support Vector Machine (Regressor)
from sklearn import linear_model

np.random.seed(42)
br = linear_model.BayesianRidge()
br.fit(x_train, y_train)

BayesianRidge()

In [32]:
# Testing Model: Score returns the coefficient of determination R^2 of the prediction. 
# The best possible score is 1.0
np.random.seed(42)
br_confidence = br.score(x_test, y_test)
print("bayesian ridge confidence: ", br_confidence)

bayesian ridge confidence:  0.8819142024916982


In [33]:
# Create and train the Linear Regression  Model
np.random.seed(42)
lr = LinearRegression()
# Train the model
lr.fit(x_train, y_train)

LinearRegression()

In [34]:
# Testing Model: Score returns the coefficient of determination R^2 of the prediction. 
# The best possible score is 1.0
lr_confidence = lr.score(x_test, y_test)
print("lr confidence: ", lr_confidence)

lr confidence:  0.8801930271195619


In [35]:
# Set x_forecast equal to the last 30 rows of the original data set from Adj. Close column
x_forecast = np.array(df.drop(['Prediction'],1))[-forecast_out:]
print(x_forecast)

[[3.05800000e+01 1.60401594e+18]
 [3.33200000e+01 1.60427515e+18]
 [3.55000000e+01 1.60436160e+18]]


In [36]:
# Print linear regression model predictions for the next '30' days
lr_prediction = lr.predict(x_forecast)
print(lr_prediction[-1])
# Print support vector regressor model predictions for the next '30' days
br_prediction = br.predict(x_forecast)
print(br_prediction[-1])

rfr_prediction = rfr.predict(x_forecast)
print(rfr_prediction[-1])

34.84982104752385
34.60425475091995
32.91778


In [37]:
br_prediction

array([30.73134608, 32.9216488 , 34.60425475])

In [38]:
lr_prediction

array([30.85013381, 33.10776927, 34.84982105])

In [39]:
rfr_prediction

array([32.91778, 32.91778, 32.91778])